In [ ]:
import hvplot.xarray
import numpy as np
import scipy.stats
import xarray as xr

import climepi
import climepi.climdata as climdata
import climepi.epimod as epimod

In [ ]:
ds_clim = climdata.get_example_dataset("isimip_london")
climepi_modes = ds_clim.climepi.modes
ds_clim = (
    ds_clim.drop_vars(["lon_bnds", "lat_bnds"])
    .squeeze()
    .sel({"scenario": ["ssp126", "ssp370", "ssp585"]})
)
ds_clim.climepi.modes = climepi_modes

In [ ]:
ds_clim_ym = ds_clim.climepi.yearly_average()
ds_clim_ym.climepi.plot_time_series("temperature", by=["scenario"])

In [ ]:
ecol_niche_model = epimod.ecolniche.get_kaye_model()
ds_clim.epimod.model = ecol_niche_model
ds_epi = ds_clim.epimod.run_model()
ds_months_suitable = ds_epi.epimod.months_suitable()
ds_months_suitable.climepi.plot_time_series(by=["scenario"])

In [ ]:
ds_ym = xr.merge([ds_clim_ym, ds_months_suitable])
ds_ym

In [ ]:
ds_ym_stat = ds_ym.climepi.estimate_ensemble_stats()
ds_ym_stat = ds_ym_stat[ds_ym_stat.climepi.get_non_bnd_data_vars()]

In [ ]:
variance_internal = ds_ym_stat.sel(ensemble_statistic="var", drop=True).mean(
    dim=["scenario", "model"]
)
variance_internal.load()

In [ ]:
variance_model = (
    ds_ym_stat.sel(ensemble_statistic="mean", drop=True)
    .var(dim="model")
    .mean(dim="scenario")
)
variance_model.load()

In [ ]:
variance_scenario = (
    ds_ym_stat.sel(ensemble_statistic="mean", drop=True)
    .mean(dim="model")
    .var(dim="scenario")
)
variance_scenario.load()

In [ ]:
check = ds_ym_stat.sel(ensemble_statistic="mean").var(dim=["scenario", "model"]) - (
    variance_model + variance_scenario
)
check.load()

In [ ]:
ds_variances = xr.concat(
    [variance_internal, variance_scenario, variance_model],
    dim=xr.Variable("variance_type", ["internal", "scenario", "model"]),
    coords="minimal",
)
ds_variances

In [ ]:
def make_variance_area_plot(_ds_variance, var, proportions=False):
    if proportions:
        _ds_variance = _ds_variance / _ds_variance.sum(dim="variance_type")
    ds_new = xr.Dataset(
        {
            "internal": _ds_variance[var].sel(variance_type="internal", drop=True),
            "scenario": _ds_variance[var].sel(variance_type="scenario", drop=True),
            "model": _ds_variance[var].sel(variance_type="model", drop=True),
        }
    )
    return ds_new.squeeze().hvplot.area(
        x="time", y=["scenario", "model", "internal"], group_label="Uncertainty type"
    )

In [ ]:
def make_plume_plot(
    _ds,
    _ds_fit,
    _ds_variances,
    var,
    conf_level=90,
    scenario_baseline=None,
    model_baseline=None,
):
    if scenario_baseline == "mean" and model_baseline == "mean":
        da_baseline = _ds_fit[var].mean(dim=["model", "scenario"])
        da_fit_scenario_model_baseline = da_baseline
        da_fit_scenario_baseline_model_mean = da_baseline
    elif scenario_baseline == "mean":
        raise ValueError(
            "If scenario_baseline is 'mean', model_baseline must be 'mean'"
        )
    elif model_baseline == "mean":
        da_baseline = _ds_fit[var].sel(scenario=scenario_baseline).mean(dim="model")
        da_fit_scenario_model_baseline = da_baseline
        da_fit_scenario_baseline_model_mean = da_baseline
    else:
        da_baseline = _ds[var].sel(scenario=scenario_baseline, model=model_baseline)
        da_fit_scenario_model_baseline = _ds_fit[var].sel(
            scenario=scenario_baseline, model=model_baseline
        )
        da_fit_scenario_baseline_model_mean = (
            _ds_fit[var].sel(scenario=scenario_baseline).mean(dim="model")
        )
    da_variances = _ds_variances[var]
    da_std_internal = np.sqrt(da_variances.sel(variance_type="internal"))
    if scenario_baseline == "mean":
        da_std_internal_model = np.sqrt(
            da_variances.sel(variance_type=["internal", "model"]).sum(
                dim="variance_type"
            )
        )
    else:
        da_std_internal_model = np.sqrt(
            _ds_fit[var].sel(scenario=scenario_baseline).var(dim="model")
            + da_variances.sel(variance_type="internal")
        )
    z = scipy.stats.norm.ppf(0.5 + conf_level / 200)
    ds_internal = xr.Dataset(
        {
            "low": da_fit_scenario_model_baseline - z * da_std_internal,
            "high": da_fit_scenario_model_baseline + z * da_std_internal,
        }
    )
    ds_model = xr.Dataset(
        {
            "low": da_fit_scenario_baseline_model_mean - z * da_std_internal_model,
            "high": da_fit_scenario_baseline_model_mean + z * da_std_internal_model,
        }
    )
    da_fit_model_scenario_mean = _ds_fit[var].mean(dim=["model", "scenario"])
    da_std_internal_model_scenario = np.sqrt(da_variances.sum(dim="variance_type"))
    ds_scenario = xr.Dataset(
        {
            "low": da_fit_model_scenario_mean - z * da_std_internal_model_scenario,
            "high": da_fit_model_scenario_mean + z * da_std_internal_model_scenario,
        }
    )
    p_baseline = da_baseline.hvplot.line(color="k", label="Baseline")
    p_internal = ds_internal.hvplot.area(x="time", y="low", y2="high", label="Internal")
    p_model = ds_model.hvplot.area(x="time", y="low", y2="high", label="Model")
    p_scenario = ds_scenario.hvplot.area(x="time", y="low", y2="high", label="Scenario")
    return p_scenario * p_model * p_internal * p_baseline

In [ ]:
data_var_plot = "months_suitable"
scenario_baseline = "mean"
model_baseline = "mean"
# scenario_baseline = "ssp370"
# model_baseline = "gfdl-esm4"

In [ ]:
p1 = ds_ym_stat.sel(ensemble_statistic="mean", drop=True).climepi.plot_time_series(
    data_var_plot, by=["scenario"]
)
p2 = ds_ym.climepi.plot_time_series(data_var_plot, by=["scenario"])
p1 * p2

In [ ]:
make_variance_area_plot(ds_variances, data_var_plot, proportions=False)

In [ ]:
make_variance_area_plot(ds_variances, "temperature", proportions=False)

In [ ]:
ds_ym.climepi.plot_var_decomp(data_var=data_var_plot)

In [ ]:
make_plume_plot(
    ds_ym.squeeze(),
    ds_ym_stat.sel(ensemble_statistic="mean", drop=True).squeeze(),
    ds_variances.squeeze(),
    data_var_plot,
    conf_level=90,
    scenario_baseline=scenario_baseline,
    model_baseline=model_baseline,
)